In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv


In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

In [52]:
X_full = pd.read_csv('../input/playground-series-s5e10/train.csv')
X_test = pd.read_csv('../input/playground-series-s5e10/test.csv')
X_full.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [53]:
X_full = X_full.drop_duplicates()

In [54]:
bool_val = ['road_signs_present', 'public_road', 'holiday', 'school_season']
X_full[bool_val] = X_full[bool_val].astype(int)
X_full.head()

X_test[bool_val] = X_test[bool_val].astype(int)

In [55]:
X_full['base_risk'] = (
        0.3 * X_full["curvature"] + 
        0.2 * (X_full["lighting"] == "night").astype(int) + 
        0.1 * (X_full["weather"] != "clear").astype(int) + 
        0.2 * (X_full["speed_limit"] >= 46).astype(int) + 
        0.1 * (np.array(X_full["num_reported_accidents"]) > 2).astype(int) +
        0.0001569
    )
X_test['base_risk'] = (
        0.3 * X_test["curvature"] + 
        0.2 * (X_test["lighting"] == "night").astype(int) + 
        0.1 * (X_test["weather"] != "clear").astype(int) + 
        0.2 * (X_test["speed_limit"] >= 46).astype(int) + 
        0.1 * (np.array(X_test["num_reported_accidents"]) > 2).astype(int) -
        0.0001569
)

In [56]:
y = X_full.pop('accident_risk')
X_full.pop('id')
ids = X_test.pop('id')

In [57]:
oh_cols = ['road_type', 'lighting', 'weather', 'time_of_day']

In [58]:
cols = list(X_full.columns)
for i in oh_cols:
    cols.remove(i)
for i in cols:
    cols.remove(i)
    for j in cols:
        X_full[f'{i}+{j}'] = X_full[i] + X_full[j]
        X_full[f'{i}*{j}'] = X_full[i] * X_full[j]
        X_test[f'{i}+{j}'] = X_test[i] + X_test[j]
        X_test[f'{i}*{j}'] = X_test[i] * X_test[j]
 
X_full.head()

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,...,school_season+base_risk,school_season*base_risk,base_risk+curvature,base_risk*curvature,base_risk+road_signs_present,base_risk*road_signs_present,base_risk+holiday,base_risk*holiday,base_risk+num_reported_accidents,base_risk*num_reported_accidents
0,urban,2,0.06,35,daylight,rainy,0,1,afternoon,0,...,1.118157,0.118157,0.178157,0.007089,0.118157,0.000000,0.118157,0.000000,1.118157,0.118157
1,urban,4,0.99,35,daylight,clear,1,0,evening,1,...,1.297157,0.297157,1.287157,0.294185,1.297157,0.297157,1.297157,0.297157,0.297157,0.000000
2,rural,4,0.63,70,dim,clear,0,1,morning,1,...,0.389157,0.000000,1.019157,0.245169,0.389157,0.000000,1.389157,0.389157,2.389157,0.778314
3,highway,4,0.07,35,dim,rainy,1,1,morning,0,...,0.121157,0.000000,0.191157,0.008481,1.121157,0.121157,0.121157,0.000000,1.121157,0.121157
4,rural,1,0.58,60,daylight,foggy,0,0,evening,1,...,0.474157,0.000000,1.054157,0.275011,0.474157,0.000000,1.474157,0.474157,1.474157,0.474157


In [59]:
OH = OneHotEncoder(sparse_output = False)
oh_val = pd.DataFrame(OH.fit_transform(X_full[oh_cols]))
new_cols = list(oh_val.columns.astype(str))
X_full[new_cols] = oh_val
X_full.head()

oh_val = pd.DataFrame(OH.transform(X_test[oh_cols]))
new_cols = list(oh_val.columns.astype(str))
X_test[new_cols] = oh_val

temp_col = ['num_lanes', 'curvature', 'speed_limit', 'num_reported_accidents']
temp_col.extend(bool_val)
temp_col.extend(new_cols)
X = X_full[temp_col]
y = X_full.accident_risk
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X))
X.columns = X.columns.astype(str)
X.head()

X.tail()

X_t, X_v, y_t, y_v = train_test_split(X, y, test_size = 0.2, random_state = 42)

len(X_t)

model = KNeighborsRegressor(100, algorithm = 'ball_tree', weights = 'uniform')

model.fit(X_t, y_t)

pred = model.predict(X_v)
mean_squared_error(pred, y_v, squared = False)

model_temp = LinearRegression()

model_temp.fit(X_t, y_t)

pred_temp = model_temp.predict(X_v)
mean_squared_error(pred_temp, y_v, squared = False)

In [60]:
print(X_full.head(1))
print(X_test.head(1))

  road_type  num_lanes  curvature  speed_limit  lighting weather  \
0     urban          2       0.06           35  daylight   rainy   

   road_signs_present  public_road time_of_day  holiday  ...    2    3    4  \
0                   0            1   afternoon        0  ...  1.0  1.0  0.0   

     5    6    7    8    9   10   11  
0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  

[1 rows x 85 columns]
  road_type  num_lanes  curvature  speed_limit lighting weather  \
0   highway          2       0.34           45    night   clear   

   road_signs_present  public_road time_of_day  holiday  ...    2    3    4  \
0                   1            1   afternoon        1  ...  0.0  0.0  0.0   

     5    6    7    8    9   10   11  
0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  

[1 rows x 85 columns]


In [61]:
diff = 0.0001569

In [62]:
X_full.drop(oh_cols, axis = 1, inplace = True)
X_test.drop(oh_cols, axis = 1, inplace = True)


In [63]:
X_full.head()

,num_lanes,curvature,speed_limit,road_signs_present,public_road,holiday,school_season,num_reported_accidents,base_risk,num_lanes+curvature,...,2,3,4,5,6,7,8,9,10,11
0,2,0.06,35,0,1,0,1,1,0.118157,2.06,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,4,0.99,35,1,0,1,1,0,0.297157,4.99,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,4,0.63,70,0,1,1,0,2,0.389157,4.63,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,4,0.07,35,1,1,0,0,1,0.121157,4.07,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1,0.58,60,0,0,1,0,1,0.474157,1.58,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [64]:
X_train, X_val, y_train, y_val = train_test_split(X_full, y, test_size = 0.2, random_state = 42)

In [65]:
from xgboost import XGBRegressor
f = ['curvature', 'base_risk', 'public_road+curvature', 'speed_limit*curvature', 
     'num_reported_accidents', 'speed_limit+curvature', 'base_risk*num_reported_accidents',
     'base_risk+holiday', 'base_risk+num_reported_accidents', 'num_lanes+curvature']

In [66]:
import cupy as cp

In [67]:
"""
i = 0.00001
for j in range(60):
    model = XGBRegressor(n_estimators = 179, max_depth = 7, 
                         max_leaves = 65, reg_lambda = 5.4, booster = 'dart', 
                         reg_alpha = 0.8, gamma = 0.00001,
                         device = 'cuda', eval_metric = 'rmse', learning_rate = 0.136, 
                         base_score = 0.34, random_state = 42 )
    score = -1 * cross_val_score(model, X_full[f], y, cv = 5, scoring = 'neg_mean_squared_error')
    print((score ** 0.5).mean(), " : ", i)
    i+=0.000005
"""

'\ni = 0.00001\nfor j in range(60):\n    model = XGBRegressor(n_estimators = 179, max_depth = 7, \n                         max_leaves = 65, reg_lambda = 5.4, booster = \'dart\', \n                         reg_alpha = 0.8, gamma = 0.00001,\n                         device = \'cuda\', eval_metric = \'rmse\', learning_rate = 0.136, \n                         base_score = 0.34, random_state = 42 )\n    score = -1 * cross_val_score(model, X_full[f], y, cv = 5, scoring = \'neg_mean_squared_error\')\n    print((score ** 0.5).mean(), " : ", i)\n    i+=0.000005\n'

In [68]:
model = XGBRegressor(n_estimators = 179, max_depth = 7, 
                         max_leaves = 65, reg_lambda = 5.4, booster = 'dart', 
                         reg_alpha = 0.8, gamma = 0.00001,
                         device = 'cuda', eval_metric = 'rmse', learning_rate = 0.136, 
                         base_score = 0.34, random_state = 42 )
#model.fit(X_full, y)

#importance = model.feature_importances_

In [69]:
"""import matplotlib.pyplot as plt
from xgboost import plot_importance
plot_importance(model, max_num_features=10)
plt.show()
f = ['curvature', 'base_risk', 'public_road+curvature', 'speed_limit*curvature', 
     'num_reported_accidents', 'speed_limit+curvature', 'base_risk*num_reported_accidents',
     'base_risk+holiday', 'base_risk+num_reported_accidents', 'num_lanes+curvature']"""

"import matplotlib.pyplot as plt\nfrom xgboost import plot_importance\nplot_importance(model, max_num_features=10)\nplt.show()\nf = ['curvature', 'base_risk', 'public_road+curvature', 'speed_limit*curvature', \n     'num_reported_accidents', 'speed_limit+curvature', 'base_risk*num_reported_accidents',\n     'base_risk+holiday', 'base_risk+num_reported_accidents', 'num_lanes+curvature']"

In [70]:
#model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_val)

In [71]:
mean_squared_error(pred, y_val, squared = False)

0.05621151604555162

In [72]:
model.fit(X_full, y)
final = model.predict(X_test)

In [73]:
output = pd.DataFrame({'id': ids, 'accident_risk':final})
output.to_csv("submission.csv", index = False)
output.head()

,id,accident_risk
0,517754,0.291141
1,517755,0.122944
2,517756,0.182802
3,517757,0.304622
4,517758,0.379375
